In [34]:
import boto3

In [35]:
s3 = boto3.resource('s3',
 aws_access_key_id='',
 aws_secret_access_key=''
)

In [53]:
s3.create_bucket(Bucket='prayush-luitel-1', CreateBucketConfiguration={
 'LocationConstraint': 'us-east-2'})


s3.Bucket(name='prayush-luitel-1')

In [54]:
bucket = s3.Bucket('prayush-luitel-1')

In [55]:
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '',
  'HostId': '5RN1UCcaM5bvkPsa87hBJE+l73qhNfHlZjf5JVk2Xp/0ZHa6Wi17beE1SUrEJQ3Pr6GkpGnrLCg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '5RN1UCcaM5bvkPsa87hBJE+l73qhNfHlZjf5JVk2Xp/0ZHa6Wi17beE1SUrEJQ3Pr6GkpGnrLCg=',
   'x-amz-request-id': '0AMVR4NZ9YK6C5WV',
   'date': 'Mon, 08 Mar 2021 21:46:20 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1}}

In [136]:
body = open('C:\\Users\\luite\\Desktop\\Cloud_HW\\Cloud_HW\\exp1.csv','rb')

In [137]:
o = s3.Object('prayush-luitel-1', 'exp1.csv').put(Body=body )

In [138]:
s3.Object('prayush-luitel-1', 'exp1.csv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KTCS5WMPe8ZJoFI0CEG+TTvzoSIGxYSEPHo2+m2knOROi5DpIAKj0uyE4bc3JvKD2v12Lsuyajk=',
   'x-amz-request-id': 'STENAABWF8FFJQ7A',
   'date': 'Tue, 09 Mar 2021 04:35:32 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [171]:
dyndb = boto3.resource('dynamodb',
 region_name='us-east-2',
 aws_access_key_id='Q',
 aws_secret_access_key=''
)

In [175]:
try:
 table = dyndb.create_table(
 TableName='Prayush_Table',
 KeySchema=[
 {
 'AttributeName': 'PartitionKey',
 'KeyType': 'HASH'
 },
 {
 'AttributeName': 'RowKey',
 'KeyType': 'RANGE'
 }
 ],
 AttributeDefinitions=[
 {
 'AttributeName': 'PartitionKey',
 'AttributeType': 'S'
 },
 {
 'AttributeName': 'RowKey',
 'AttributeType': 'S'
 },
 ],
 ProvisionedThroughput={
 'ReadCapacityUnits': 5,
 'WriteCapacityUnits': 5
 }
 )
except:
 #if there is an exception, the table may already exist. if so...
 table = dyndb.Table("Prayush_Table")


In [176]:
table.meta.client.get_waiter('table_exists').wait(TableName='Prayush_Table')

In [177]:
print(table.item_count)

0


In [178]:
import csv

In [179]:
path = 'C:\\Users\\luite\\Desktop\\Cloud_HW\\Cloud_HW\\experiments.csv'

In [183]:
with open(path, 'r') as csvfile:
 csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
 for item in csvf:
     print (item)
     body = open('C:\\Users\\luite\\Desktop\\Cloud_HW\\Cloud_HW\\'+item[4], 'rb')
     s3.Object('prayush-luitel-1', item[4]).put(Body=body)
     md = s3.Object('prayush-luitel-1', item[4]).Acl().put(ACL='public-read')

     url = " https://s3-us-east-2.amazonaws.com/prayush-luitel-1/"+item[4]
     metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
     'description' : item[3], 'date' : item[2], 'url':url}
     table.put_item(Item=metadata_item)

['experiment1', 'data1', '2/13/2021', 'Words..', 'exp1.csv']
['experiment2', 'data2', '2/20/2021', 'second experiment', 'exp2.csv']


In [185]:
response1 = table.get_item(
 Key={
 'PartitionKey': 'experiment1',
 'RowKey': 'data1'
 }
)
item = response1
print(item['Item'])

{'PartitionKey': 'experiment1', 'RowKey': 'data1', 'date': '2/13/2021', 'description': 'Words..', 'url': ' https://s3-us-east-2.amazonaws.com/prayush-luitel-1/exp1.csv'}


In [189]:
response1

{'Item': {'PartitionKey': 'experiment1',
  'RowKey': 'data1',
  'date': '2/13/2021',
  'description': 'Words..',
  'url': ' https://s3-us-east-2.amazonaws.com/prayush-luitel-1/exp1.csv'},
 'ResponseMetadata': {'RequestId': 'OAA0VQ6P8C3MGN0GM2DPT4C2OVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 09 Mar 2021 05:03:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '199',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'OAA0VQ6P8C3MGN0GM2DPT4C2OVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2226355678'},
  'RetryAttempts': 0}}

In [188]:
response2 = table.get_item(
 Key={
 'PartitionKey': 'experiment2',
 'RowKey': 'data2'
 }
)
item = response2
print(item['Item'])

{'PartitionKey': 'experiment2', 'RowKey': 'data2', 'date': '2/20/2021', 'description': 'second experiment', 'url': ' https://s3-us-east-2.amazonaws.com/prayush-luitel-1/exp2.csv'}


In [190]:
response2

{'Item': {'PartitionKey': 'experiment2',
  'RowKey': 'data2',
  'date': '2/20/2021',
  'description': 'second experiment',
  'url': ' https://s3-us-east-2.amazonaws.com/prayush-luitel-1/exp2.csv'},
 'ResponseMetadata': {'RequestId': 'P2FUMHN2EC9NT3D73QKJ7MDDM7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 09 Mar 2021 05:04:19 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '209',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'P2FUMHN2EC9NT3D73QKJ7MDDM7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1621330087'},
  'RetryAttempts': 0}}